# 브랜드별 매장 데이터 전처리

### 목표 
 - 브랜드별 크롤링 결과를 묶어서 데이터 분석할 수 있는 여러 데이터프레임을 생성 및 저장한다. 

### 주요 내용
- 컬럼명 획일화 (예) "지점명", "매장명" → "지점명"으로
- 주소 행정구역명 획일화 (예) "서울시","서울특별시","서울" → "서울특별시"로

### 최종 데이터프레임 
#### 1. `burger_df`
    - 크기 : (2306, 6)
    - 파일명 : *burger_df.csv* 또는 *burger_df.xlsx*로 저장
    - 구성  
    
|index|브랜드|주소|지점명|주소1(시,도)|주소2(시,구,군)|주소3(나머지)|
|---|---|---|---|---|---|---|
|0|버거킹| 서울특별시 강남구 선릉로 429	| 선릉역점	| 서울특별시| 강남구 | 선릉로 429|
|...|...|...|...|...|...|...|...|

#### 2. `nstore_df`
    - 크기 : (240, 9)
    - 파일명 : *nstore_df.csv* 또는 *nstore_df.xlsx*
    - 구성
|index|버거킹|KFC|맥도날드|롯데리아|BKM|버거지수|주소(주소1+주소2)|주소1(시,도)|주소2(시,구,군)|
|---|---|---|---|---|---|---|---|---|---|
|0|1| 0	|1| 6|2 | 0.3333| 강원도 강릉시| 강원도|강릉시|
|...|...|...|...|...|...|...|...|...|...|  
    	
#### 3. `nstore_stat_df`
    - 크기 :(17, 13)
    - 파일명 : *nstore_stat_df.csv* 또는 *nstore_stat_df.xlsx*
    - 구성 
    cf. 경제 지표 기준은 1인당

|index|버거킹|KFC|맥도날드|롯데리아|BKM|버거지수|시도별|지역내총생산|지역총소득| 개인소득|민간소비|인구|인구밀도|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|0|1| 0	|1| 6|2 | 0.3333| 강원도 |32061|29392|18997|16811|1521|90|
|...|...|...|...|...|...|...|...| ...| ...|  ...| ...| ...| ...|  

    

### 특이사항
- 롯데리아의 경우 크롤링한 결과 자체가 `burger_df` 과 동일한 구성으로 되어있기 때문에 나머지 브랜드를 처리한 후에 `pandas.concat` 이용하여 추가
   


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import folium
from folium import plugins
import json

import plotly
import cufflinks as cf
import plotly.express as px

# 오프라인 모드에서도 인터렉티브한 그래픽을 가능하도록 함
from plotly.offline import download_plotlyjs,init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

# 다중 그래프 그리기
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# output 파일 path 설정
output_path = './output/'
input_path = './input/'

In [3]:
# 각 브랜드별 DataFrame 생성
burger_king_df = pd.read_excel('./BurgerKing/BurgerKing_all_store.xlsx')
kfc_df = pd.read_excel('./KFC/KFC_all_store.xlsx')
mcdonalds_df = pd.read_excel('./Mcdonalds/mcdonalds_all_store.xlsx')
lotteria_df = pd.read_excel('./Lotteria/lotteria_all_store.xlsx')

In [4]:
burger_king_df.head()

,지점명,주소
0,선릉역점,서울특별시 강남구 선릉로 429
1,대치역점,서울특별시 강남구 남부순환로 2936
2,차병원사거리점,서울특별시 강남구 봉은사로 179
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층
4,청담점,서울특별시 강남구 선릉로 812


In [5]:
kfc_df.head()

,매장명,매장 주소
0,원주단계DT,강원 원주시 북원로 2266 (단계동) KFC원주단계DT
1,춘천이마트,강원 춘천시 경춘로 2341 (온의동) 이마트 춘천점 1층 푸드코트 內
2,춘천석사,강원 춘천시 영서로 2027 (석사동)
3,행신역,경기 고양시 덕양구 충장로 8 (행신동)
4,화정,경기 고양시 덕양구 화신로272번길 57 (화정동)


In [6]:
mcdonalds_df.head()

,매장명,매장 주소
0,고양 삼송 DT,경기 고양시 덕양구 고양대로 1948
1,서울시청,서울 중구 남대문로9길 51 효덕빌딩
2,한신,서울 노원구 한글비석로 57 (하계동)
3,영천DT,경북 영천시 호국로 141
4,대구동호DT,대구 동구 안심로 403


In [7]:
lotteria_df.head()

,지점명,주소,주소1,주소2,주소3
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5


In [8]:
burger_king_df.shape, kfc_df.shape, mcdonalds_df.shape, lotteria_df.shape

((423, 2), (187, 2), (405, 2), (1305, 5))

In [9]:
# 각 DataFrame에 브랜드 열 생성
burger_king_df['브랜드'] = '버거킹'
kfc_df['브랜드'] = 'KFC'
mcdonalds_df['브랜드'] = '맥도날드'
lotteria_df['브랜드'] = '롯데리아'

In [10]:
# 중복 데이터 확인
print( burger_king_df['지점명'].value_counts().sort_values(ascending=False).head(1) )
print()
print( kfc_df['매장명'].value_counts().sort_values(ascending=False).head(1) )
print()
print( mcdonalds_df['매장명'].value_counts().sort_values(ascending=False).head(1) )
print()
print( lotteria_df['지점명'].value_counts().sort_values(ascending=False).head(1) )

인천구월아시아드SK점    2
Name: 지점명, dtype: int64

일산후곡    1
Name: 매장명, dtype: int64

인천계산DT    1
Name: 매장명, dtype: int64

일산마두    1
Name: 지점명, dtype: int64


In [11]:
# 중복 제거
burger_king_df.drop_duplicates('지점명', keep='first', inplace=True)
kfc_df.drop_duplicates('매장명', keep='first', inplace=True)
mcdonalds_df.drop_duplicates('매장명', keep='first', inplace=True)
lotteria_df.drop_duplicates('지점명', keep='first', inplace=True)

burger_king_df.shape, kfc_df.shape, mcdonalds_df.shape, lotteria_df.shape

((410, 3), (187, 3), (405, 3), (1305, 6))

In [12]:
# 빈 DataFrame 생성 후 각 브랜드별 DataFrame 추가
burger_df = pd.DataFrame()

burger_df = pd.concat([burger_king_df,kfc_df, mcdonalds_df])
# burger_df = pd.concat([burger_df, lotteria_df])

# DataFrame 인덱스 재설정
burger_df.reset_index(drop=True, inplace=True)

In [13]:
print(burger_df.shape)
burger_df.head()

(1002, 5)


,지점명,주소,브랜드,매장명,매장 주소
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,NaN,NaN
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,NaN,NaN
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,NaN,NaN
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,NaN,NaN
4,청담점,서울특별시 강남구 선릉로 812,버거킹,NaN,NaN


In [14]:
# 매장명, 매장 주소를 지점명, 주소로 통일
burger_df['지점명'].fillna(burger_df['매장명'], inplace=True)
burger_df['주소'].fillna(burger_df['매장 주소'], inplace=True)
print(burger_df.isnull().sum())
burger_df.head()

지점명        0
주소         0
브랜드        0
매장명      410
매장 주소    410
dtype: int64


,지점명,주소,브랜드,매장명,매장 주소
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,NaN,NaN
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,NaN,NaN
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,NaN,NaN
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,NaN,NaN
4,청담점,서울특별시 강남구 선릉로 812,버거킹,NaN,NaN


In [15]:
# 매장명, 매장 주소 열 삭제
burger_df.drop(['매장명', '매장 주소'], axis=1, inplace=True)
burger_df

,지점명,주소,브랜드
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹
4,청담점,서울특별시 강남구 선릉로 812,버거킹
...,...,...,...
997,대전카이스트점,대전 유성구 대덕대로 535,맥도날드
998,김천평화DT점,경북 김천시 자산로 199,맥도날드
999,대구태전 DT,대구 북구 칠곡중앙대로 303,맥도날드
1000,강남 2호점,서울 강남구 테헤란로 107 메디타워2층,맥도날드


In [16]:
# 주소를 시군구별로 나눠서 열 추가 후 재정렬
burger_df[['주소1', '주소2', '주소3']] = pd.DataFrame(burger_df['주소'].str.strip().str.split(' ', 2).tolist())
burger_df.reindex(columns=['브랜드', '지점명', '주소', '주소1', '주소2', '주소3'])

,브랜드,지점명,주소,주소1,주소2,주소3
0,버거킹,선릉역점,서울특별시 강남구 선릉로 429,서울특별시,강남구,선릉로 429
1,버거킹,대치역점,서울특별시 강남구 남부순환로 2936,서울특별시,강남구,남부순환로 2936
2,버거킹,차병원사거리점,서울특별시 강남구 봉은사로 179,서울특별시,강남구,봉은사로 179
3,버거킹,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층
4,버거킹,청담점,서울특별시 강남구 선릉로 812,서울특별시,강남구,선릉로 812
...,...,...,...,...,...,...
997,맥도날드,대전카이스트점,대전 유성구 대덕대로 535,대전,유성구,대덕대로 535
998,맥도날드,김천평화DT점,경북 김천시 자산로 199,경북,김천시,자산로 199
999,맥도날드,대구태전 DT,대구 북구 칠곡중앙대로 303,대구,북구,칠곡중앙대로 303
1000,맥도날드,강남 2호점,서울 강남구 테헤란로 107 메디타워2층,서울,강남구,테헤란로 107 메디타워2층


In [17]:
# 시도 값 확인
burger_df['주소1'].unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도',
       '제주특별자치도', '강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울',
       '세종특별자치시', '울산', '인천', '전남', '전북', '충남', '충북', '서울시'], dtype=object)

In [18]:
# 줄여쓴 구역을 풀어서 다시 입력
burger_df['주소1'] = burger_df['주소1'].replace({'서울시':'서울특별시',
                                                '서울':'서울특별시',
                                                '부산':'부산광역시',
                                                '대구':'대구광역시',
                                                '인천':'인천광역시',
                                                '광주':'광주광역시',
                                                '대전':'대전광역시',
                                                '울산':'울산광역시',
                                                '세종':'세종특별자치시',
                                                '경기':'경기도',
                                                '강원':'강원도',
                                                '충북':'충청북도',
                                                '충남':'충청남도',
                                                '전북':'전라북도',
                                                '전남':'전라남도',
                                                '경북':'경상북도',
                                                '경남':'경상남도',
                                                '제주':'제주특별자치도'})

burger_df

,지점명,주소,브랜드,주소1,주소2,주소3
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,서울특별시,강남구,선릉로 429
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,서울특별시,강남구,남부순환로 2936
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,서울특별시,강남구,봉은사로 179
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층
4,청담점,서울특별시 강남구 선릉로 812,버거킹,서울특별시,강남구,선릉로 812
...,...,...,...,...,...,...
997,대전카이스트점,대전 유성구 대덕대로 535,맥도날드,대전광역시,유성구,대덕대로 535
998,김천평화DT점,경북 김천시 자산로 199,맥도날드,경상북도,김천시,자산로 199
999,대구태전 DT,대구 북구 칠곡중앙대로 303,맥도날드,대구광역시,북구,칠곡중앙대로 303
1000,강남 2호점,서울 강남구 테헤란로 107 메디타워2층,맥도날드,서울특별시,강남구,테헤란로 107 메디타워2층


In [19]:
# 주소1, 주소2, 주소3 을 합쳐서 주소를 다시 입력
burger_df['주소'] = burger_df[ ['주소1', '주소2', '주소3'] ].apply(' '.join, axis=1)
burger_df

,지점명,주소,브랜드,주소1,주소2,주소3
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,서울특별시,강남구,선릉로 429
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,서울특별시,강남구,남부순환로 2936
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,서울특별시,강남구,봉은사로 179
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층
4,청담점,서울특별시 강남구 선릉로 812,버거킹,서울특별시,강남구,선릉로 812
...,...,...,...,...,...,...
997,대전카이스트점,대전광역시 유성구 대덕대로 535,맥도날드,대전광역시,유성구,대덕대로 535
998,김천평화DT점,경상북도 김천시 자산로 199,맥도날드,경상북도,김천시,자산로 199
999,대구태전 DT,대구광역시 북구 칠곡중앙대로 303,맥도날드,대구광역시,북구,칠곡중앙대로 303
1000,강남 2호점,서울특별시 강남구 테헤란로 107 메디타워2층,맥도날드,서울특별시,강남구,테헤란로 107 메디타워2층


In [20]:
lotteria_df.head()

,지점명,주소,주소1,주소2,주소3,브랜드
0,가평휴게소(상),경기도 가평군 설악면 미사리로 544,경기도,가평군,설악면 미사리로 544,롯데리아
1,아산장재,서울특별시 용산구 한강대로71길 47,서울특별시,용산구,한강대로71길 47,롯데리아
2,수원파장,경기도 수원시 장안구 파장로 91,경기도,수원시,장안구 파장로 91,롯데리아
3,롯데더몰여수,전라남도 여수시 국포1로 36,전라남도,여수시,국포1로 36,롯데리아
4,김포마산,경기도 김포시 김포한강8로148번길 5,경기도,김포시,김포한강8로148번길 5,롯데리아


In [21]:
# 롯데리아 데이터 추가
burger_df = pd.concat([burger_df, lotteria_df])
burger_df.reset_index(drop=True, inplace=True)
burger_df

,지점명,주소,브랜드,주소1,주소2,주소3
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,서울특별시,강남구,선릉로 429
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,서울특별시,강남구,남부순환로 2936
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,서울특별시,강남구,봉은사로 179
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층
4,청담점,서울특별시 강남구 선릉로 812,버거킹,서울특별시,강남구,선릉로 812
...,...,...,...,...,...,...
2302,원주일산,강원도 원주시 천사로 213,롯데리아,강원도,원주시,천사로 213
2303,성남,경기도 성남시 수정구 수정로 181,롯데리아,경기도,성남시,수정구 수정로 181
2304,대전중앙,대전광역시 중구 선화동 3-3번지,롯데리아,대전광역시,중구,선화동 3-3번지
2305,홈서비스과천,경기도 과천시 별양동 19-4,롯데리아,경기도,과천시,별양동 19-4


In [22]:
# csv, xlsx 파일로 내보내기
burger_df.to_csv(output_path+"burger_df.csv", index=False)
burger_df.to_excel(output_path+"burger_df.xlsx", index=False)

os.listdir(output_path)

['.DS_Store',
 'burger_df.csv',
 'burger_df.xlsx',
 'nstore_df.csv',
 'nstore_df.xlsx',
 'nstore_stat_df.csv',
 'nstore_stat_df.xlsx']

> `burger_df` 파일 전처리 완료

In [23]:
# 주소1+2 컬럼 만들기
burger_df["주소1+2"] = burger_df["주소1"]+" "+burger_df["주소2"]
burger_df

,지점명,주소,브랜드,주소1,주소2,주소3,주소1+2
0,선릉역점,서울특별시 강남구 선릉로 429,버거킹,서울특별시,강남구,선릉로 429,서울특별시 강남구
1,대치역점,서울특별시 강남구 남부순환로 2936,버거킹,서울특별시,강남구,남부순환로 2936,서울특별시 강남구
2,차병원사거리점,서울특별시 강남구 봉은사로 179,버거킹,서울특별시,강남구,봉은사로 179,서울특별시 강남구
3,강남도곡점,서울특별시 강남구 논현로 172 (도곡동 410-10) 1층,버거킹,서울특별시,강남구,논현로 172 (도곡동 410-10) 1층,서울특별시 강남구
4,청담점,서울특별시 강남구 선릉로 812,버거킹,서울특별시,강남구,선릉로 812,서울특별시 강남구
...,...,...,...,...,...,...,...
2302,원주일산,강원도 원주시 천사로 213,롯데리아,강원도,원주시,천사로 213,강원도 원주시
2303,성남,경기도 성남시 수정구 수정로 181,롯데리아,경기도,성남시,수정구 수정로 181,경기도 성남시
2304,대전중앙,대전광역시 중구 선화동 3-3번지,롯데리아,대전광역시,중구,선화동 3-3번지,대전광역시 중구
2305,홈서비스과천,경기도 과천시 별양동 19-4,롯데리아,경기도,과천시,별양동 19-4,경기도 과천시


In [24]:
# 브랜드별로 데이터 묶기
bking_grp = burger_df[burger_df["브랜드"]=="버거킹"]
kfc_grp = burger_df[burger_df["브랜드"]=="KFC"]
mc_grp = burger_df[burger_df["브랜드"]=="맥도날드"]
lotte_grp = burger_df[burger_df["브랜드"]=="롯데리아"]

bking_grp.shape, kfc_grp.shape, mc_grp.shape, lotte_grp.shape # 갯수 확인

((410, 7), (187, 7), (405, 7), (1305, 7))

In [25]:
# 시구군별로 매장수 세기
bking_cnt =  bking_grp.groupby("주소1+2").count()["지점명"]
kfc_cnt = kfc_grp.groupby("주소1+2").count()["지점명"]
mc_cnt = mc_grp.groupby("주소1+2").count()["지점명"]
lotte_cnt = lotte_grp.groupby("주소1+2").count()["지점명"]

bking_cnt.shape, kfc_cnt.shape, mc_cnt.shape, lotte_cnt.shape

((129,), (90,), (132,), (238,))

In [26]:
# column 이름을 Series.name 에 저장
bking_cnt.name = "버거킹"
kfc_cnt.name = "KFC"
mc_cnt.name = "맥도날드"
lotte_cnt.name = "롯데리아"

In [27]:
# Series 데이터 확인
print(bking_cnt)
print()
print(kfc_cnt)
print()
print(mc_cnt)
print()
print(lotte_cnt)
print()

주소1+2
강원도 강릉시     1
강원도 속초시     1
강원도 원주시     3
강원도 춘천시     2
강원도 홍천군     2
           ..
충청남도 홍성군    1
충청북도 음성군    1
충청북도 진천군    1
충청북도 청주시    7
충청북도 충주시    1
Name: 버거킹, Length: 129, dtype: int64

주소1+2
강원도 원주시         1
강원도 춘천시         2
경기도 고양시         5
경기도 광명시         2
경기도 구리시         1
               ..
제주특별자치도 서귀포시    1
충청남도 당진시        1
충청남도 아산시        1
충청남도 천안시        3
충청북도 청주시        2
Name: KFC, Length: 90, dtype: int64

주소1+2
강원도 강릉시     1
강원도 동해시     1
강원도 속초시     1
강원도 원주시     2
강원도 춘천시     2
           ..
충청남도 천안시    4
충청남도 홍성군    1
충청북도 제천시    1
충청북도 청주시    6
충청북도 충주시    1
Name: 맥도날드, Length: 132, dtype: int64

주소1+2
강원도 강릉시      6
강원도 고성군      1
강원도 동해시      3
강원도 삼척시      1
강원도 속초시      3
            ..
충청북도 증평군     1
충청북도 진천군     2
충청북도 청원군     4
충청북도 청주시    27
충청북도 충주시     8
Name: 롯데리아, Length: 238, dtype: int64



In [28]:
# Series 합치기
nstore_df = pd.concat([bking_cnt, kfc_cnt, mc_cnt, lotte_cnt], axis=1) # 시리즈 합치기, 합치면서 dtypes=float으로 변하는 것 같다
nstore_df = nstore_df.fillna(0) # 결측치 0으로 채우기 
nstore_df = nstore_df.astype("int") # 자료형 타입 int로 바꾸기

print(nstore_df.info())
nstore_df

<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, 강원도 강릉시 to 충청북도 청원군
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   버거킹     240 non-null    int32
 1   KFC     240 non-null    int32
 2   맥도날드    240 non-null    int32
 3   롯데리아    240 non-null    int32
dtypes: int32(4)
memory usage: 5.6+ KB
None


,버거킹,KFC,맥도날드,롯데리아
강원도 강릉시,1,0,1,6
강원도 속초시,1,0,1,3
강원도 원주시,3,1,2,15
강원도 춘천시,2,2,2,7
강원도 홍천군,2,0,0,2
...,...,...,...,...
충청북도 보은군,0,0,0,1
충청북도 영동군,0,0,0,1
충청북도 옥천군,0,0,0,2
충청북도 증평군,0,0,0,1


In [29]:
# 시군구 중복 확인
len(nstore_df.index), len(set(nstore_df.index)) 

(240, 240)

In [30]:
# 버거지수 만들기 전 롯데리아 매장이 없는 지역 확인 → 0으로 나누면 inf이 된다! nan으로 처리
nstore_df[nstore_df["롯데리아"]==0]

,버거킹,KFC,맥도날드,롯데리아
인천광역시 미추홀구,3,1,1,0
세종특별자치시 금송로,0,1,0,0


In [31]:
# 버거지수 컬럼 만들기
nstore_df["BKM"]=nstore_df["버거킹"]+nstore_df["KFC"]+nstore_df["맥도날드"]
nstore_df["버거지수"] = nstore_df["BKM"]/nstore_df["롯데리아"] 

# 이상치 처리
nstore_df.loc["인천광역시 미추홀구","버거지수"]= None
nstore_df.loc["세종특별자치시 금송로","버거지수"] = None
nstore_df

,버거킹,KFC,맥도날드,롯데리아,BKM,버거지수
강원도 강릉시,1,0,1,6,2,0.333333
강원도 속초시,1,0,1,3,2,0.666667
강원도 원주시,3,1,2,15,6,0.400000
강원도 춘천시,2,2,2,7,6,0.857143
강원도 홍천군,2,0,0,2,2,1.000000
...,...,...,...,...,...,...
충청북도 보은군,0,0,0,1,0,0.000000
충청북도 영동군,0,0,0,1,0,0.000000
충청북도 옥천군,0,0,0,2,0,0.000000
충청북도 증평군,0,0,0,1,0,0.000000


In [32]:
# 인덱스를 주소1(시,도), 주소2(시,군,구)로 나누기, 인덱싱 재설정은 나중에 한다
nstore_df["주소"] = nstore_df.index
nstore_df[["주소1","주소2"]] = nstore_df["주소"].str.strip().str.split().tolist()
nstore_df

,버거킹,KFC,맥도날드,롯데리아,BKM,버거지수,주소,주소1,주소2
강원도 강릉시,1,0,1,6,2,0.333333,강원도 강릉시,강원도,강릉시
강원도 속초시,1,0,1,3,2,0.666667,강원도 속초시,강원도,속초시
강원도 원주시,3,1,2,15,6,0.400000,강원도 원주시,강원도,원주시
강원도 춘천시,2,2,2,7,6,0.857143,강원도 춘천시,강원도,춘천시
강원도 홍천군,2,0,0,2,2,1.000000,강원도 홍천군,강원도,홍천군
...,...,...,...,...,...,...,...,...,...
충청북도 보은군,0,0,0,1,0,0.000000,충청북도 보은군,충청북도,보은군
충청북도 영동군,0,0,0,1,0,0.000000,충청북도 영동군,충청북도,영동군
충청북도 옥천군,0,0,0,2,0,0.000000,충청북도 옥천군,충청북도,옥천군
충청북도 증평군,0,0,0,1,0,0.000000,충청북도 증평군,충청북도,증평군


#### 주소2(시,군,구) 중복 확인
- 이 후에 주소2를 기준으로 인구나 인구밀도 정보를 넣어도 되는지 알기 위해서 중복을 확인한다.
- 결과 : 중복있음. 주소2를 기준으로 인구나 인구밀도 정보를 넣으면 안된다. 
- 대신, 인덱스 값은 중복이 없으니 기준으로 사용할 것이다. reset_index를 통해 숫자형 인덱스를 지정하고 기존의 값을 새로운 컬럼에 저장한다. 

#### cf. 중복 제거 방식 2가지
1. set 사용하여 길이 비교
2. pandas.drop_duplicates() 메서드 사용

In [33]:
# 주소2(시,군,구) 중복 확인
print(len(nstore_df['주소2']))
print(len(set(nstore_df["주소2"]))) # 중복 제거 - 1. set 사용
print(len(nstore_df["주소2"].drop_duplicates())) # 중복 제거 - 2. pandas.drop_duplicates() 사용

nstore_df['주소2'].value_counts().head(10) # 어느 도/시에서 겹치는거지? 아래의 셀에서 확인

240
218
218


중구      6
남구      5
동구      5
서구      5
북구      4
고성군     2
강서구     2
보람동     1
양산시     1
의정부시    1
Name: 주소2, dtype: int64

In [34]:
# 주소2(시,군,구) 이름이 겹치는 주소1(시,도) 
list_duplicates = list(nstore_df['주소2'].value_counts().head(6).index)
list_duplicates

for one in list_duplicates :
    print(nstore_df[nstore_df["주소2"]==one]["주소1"], "\n")

대구광역시 중구    대구광역시
대전광역시 중구    대전광역시
서울특별시 중구    서울특별시
울산광역시 중구    울산광역시
인천광역시 중구    인천광역시
부산광역시 중구    부산광역시
Name: 주소1, dtype: object 

광주광역시 남구    광주광역시
대구광역시 남구    대구광역시
부산광역시 남구    부산광역시
울산광역시 남구    울산광역시
인천광역시 남구    인천광역시
Name: 주소1, dtype: object 

광주광역시 동구    광주광역시
대구광역시 동구    대구광역시
대전광역시 동구    대전광역시
울산광역시 동구    울산광역시
부산광역시 동구    부산광역시
Name: 주소1, dtype: object 

광주광역시 서구    광주광역시
대구광역시 서구    대구광역시
대전광역시 서구    대전광역시
인천광역시 서구    인천광역시
부산광역시 서구    부산광역시
Name: 주소1, dtype: object 

광주광역시 북구    광주광역시
대구광역시 북구    대구광역시
부산광역시 북구    부산광역시
울산광역시 북구    울산광역시
Name: 주소1, dtype: object 

강원도 고성군      강원도
경상남도 고성군    경상남도
Name: 주소1, dtype: object 



#### cf. 인덱싱 재설정 `reset_index(drop=False, inplace=False)`
- drop : 기존의 인덱스 값을 삭제할 것인가?
    - False(default) : 삭제하지 않고 새로운 컬럼에 저장
    - True : 삭제
- inplace : 기존의 데이터프레임에 저장할 것인가?
    - False(default) : 기존의 데이터프레임에는 저장하지 않고 출력만 
    - True : 기존의 데이터프레임에 저장

In [35]:
# 인덱싱 재설정
nstore_df.reset_index(drop=True, inplace=True) 
nstore_df

,버거킹,KFC,맥도날드,롯데리아,BKM,버거지수,주소,주소1,주소2
0,1,0,1,6,2,0.333333,강원도 강릉시,강원도,강릉시
1,1,0,1,3,2,0.666667,강원도 속초시,강원도,속초시
2,3,1,2,15,6,0.400000,강원도 원주시,강원도,원주시
3,2,2,2,7,6,0.857143,강원도 춘천시,강원도,춘천시
4,2,0,0,2,2,1.000000,강원도 홍천군,강원도,홍천군
...,...,...,...,...,...,...,...,...,...
235,0,0,0,1,0,0.000000,충청북도 보은군,충청북도,보은군
236,0,0,0,1,0,0.000000,충청북도 영동군,충청북도,영동군
237,0,0,0,2,0,0.000000,충청북도 옥천군,충청북도,옥천군
238,0,0,0,1,0,0.000000,충청북도 증평군,충청북도,증평군


In [36]:
# csv, xlsx 파일로 내보내기
nstore_df.to_csv(output_path+"nstore_df.csv", index=False)
nstore_df.to_excel(output_path+"nstore_df.xlsx", index=False)

os.listdir(output_path)

['.DS_Store',
 'burger_df.csv',
 'burger_df.xlsx',
 'nstore_df.csv',
 'nstore_df.xlsx',
 'nstore_stat_df.csv',
 'nstore_stat_df.xlsx']

## 시도별 경제 지표 가져오기
- 출처 : [통계청](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1C86&conn_path=I2)  
- 파일명 : *시도별_1인당_지역내총생산\__지역총소득\__개인소득_20210702225439.csv* 
- 설명 : (2019년 기준) 전국 16개 시도별 `1인당 지역내총생산`, `1인당 지역총소득`,`1인당 개인소득`,`1인당 민간소비``

In [37]:
eco_index = pd.read_csv(input_path+"시도별_1인당_지역내총생산__지역총소득__개인소득_20210702225439.csv", encoding='cp949', skiprows=1)
eco_index

,시도별,1인당 지역내총생산,1인당 지역총소득,1인당 개인소득,1인당 민간소비
0,전국,37208,37530,20400,17962
1,서울특별시,44865,49121,23440,21891
2,부산광역시,27409,29388,19680,18030
3,대구광역시,23744,27798,19210,17850
4,인천광역시,30425,32571,19332,16451
5,광주광역시,27548,30964,20532,18231
6,대전광역시,28364,31548,20498,18025
7,울산광역시,65352,54969,22550,18482
8,세종특별자치시,35826,36983,19789,16762
9,경기도,36133,38466,20482,17399


In [38]:
# 행정구역 이름이 같은지 확인
print(set(eco_index["시도별"][1:]) == set(nstore_df["주소1"].unique()))

True


### 두가지 방식
경제지표가 __시군구 단위__가 아니라 __시도 단위__이기 때문에 분석 방법에 <U>두가지</U>가 있다.  
#### 1. `nstore_df`에 `eco_index`를 join하는 방식
- 어떤 구/군인가에 상관없이 동일한 시/도에 포함되면 동일한 경제지표를 가지게 된다.  
- 방식 2.보다는 구/군에 따른 데이터 수가 많기 때문에 산점도를 그렸을 때 더 세부적인 버거지수 비교가 가능하다.
- 시의 경제 지표가 그에 포함된 구의 경제지표를 대표하기엔 다소 무리가 있다. 각 시/도 안에서도 경제적인 수준의 편차가 존재하기 때문에.  

#### 2. `eco_index`에 `nstore_df`를 join하는 방식
- 해당 시/도당 하나의 버거지수
- 버거지수-경제지표 산점도에 데이터 수가 줄어든다.
- 논리적으로는 이 방식이 더 맞아보인다.

### 방식 1.  nstore_df에 eco_index를 join한다.
- 결과 : 실패

### 방식 2. `eco_index`에 `nstore_df`를 join하는 방식

In [39]:
new_nstore_df = nstore_df.groupby("주소1").sum()[["버거킹","KFC","맥도날드","롯데리아"]] 
new_nstore_df # 롯데리아가 없는 시,도가 있는지 직접 확인

,버거킹,KFC,맥도날드,롯데리아
주소1,,,,
강원도,9,3,7,54
경기도,100,50,96,317
경상남도,24,4,29,87
경상북도,25,4,15,68
광주광역시,15,3,10,42
대구광역시,20,10,25,66
대전광역시,14,5,13,51
부산광역시,20,9,41,82
서울특별시,111,73,96,181


In [40]:
new_nstore_df["BKM"]= new_nstore_df["버거킹"]+new_nstore_df["KFC"]+new_nstore_df["맥도날드"]
new_nstore_df["버거지수"] = new_nstore_df["BKM"]/new_nstore_df["롯데리아"]
new_nstore_df

,버거킹,KFC,맥도날드,롯데리아,BKM,버거지수
주소1,,,,,,
강원도,9,3,7,54,19,0.351852
경기도,100,50,96,317,246,0.776025
경상남도,24,4,29,87,57,0.655172
경상북도,25,4,15,68,44,0.647059
광주광역시,15,3,10,42,28,0.666667
대구광역시,20,10,25,66,55,0.833333
대전광역시,14,5,13,51,32,0.627451
부산광역시,20,9,41,82,70,0.853659
서울특별시,111,73,96,181,280,1.546961


In [41]:
df_2 = pd.merge(new_nstore_df, eco_index, left_on=new_nstore_df.index, right_on='시도별', how='left')
df_2

,버거킹,KFC,맥도날드,롯데리아,BKM,버거지수,시도별,1인당 지역내총생산,1인당 지역총소득,1인당 개인소득,1인당 민간소비
0,9,3,7,54,19,0.351852,강원도,32061,29392,18997,16811
1,100,50,96,317,246,0.776025,경기도,36133,38466,20482,17399
2,24,4,29,87,57,0.655172,경상남도,33690,32140,18939,16426
3,25,4,15,68,44,0.647059,경상북도,40272,34873,18611,16217
4,15,3,10,42,28,0.666667,광주광역시,27548,30964,20532,18231
5,20,10,25,66,55,0.833333,대구광역시,23744,27798,19210,17850
6,14,5,13,51,32,0.627451,대전광역시,28364,31548,20498,18025
7,20,9,41,82,70,0.853659,부산광역시,27409,29388,19680,18030
8,111,73,96,181,280,1.546961,서울특별시,44865,49121,23440,21891
9,0,1,0,10,1,0.100000,세종특별자치시,35826,36983,19789,16762


## 시도별 인구수/인구밀도 데이터 가져오기
- 출처 : 시도별 인구수/인구밀도 [통계청](https://www.index.go.kr/potal/stts/idxMain/selectPoSttsIdxSearch.do?idx_cd=1007&stts_cd=100701&freq=Y)
- 파일명: *stat_100701.xls*
- 설명 : (2020년 기준) 전국 16개 시도별 `인구`[천명], `인구밀도`[명/km^2]

In [42]:
pop_df_all = pd.read_excel(input_path+"stat_100701.xls",skiprows=3)
pop_df = pop_df_all[1:-5] # 불필요한 행 제거
pop_df = pop_df[["Unnamed: 0", "인구.1","인구밀도.1"]] # 불필요한 열 제거
pop_df = pop_df.rename(columns={"Unnamed: 0": "행정구역", "인구.1":"인구","인구밀도.1":"인구밀도"}) # 컬럼명 수정
# 행정구역 이름 동일화
pop_df = pop_df.replace({'서울':'서울특별시',
                        '부산':'부산광역시',
                        '대구':'대구광역시',
                        '인천':'인천광역시',
                        '광주':'광주광역시',
                        '대전':'대전광역시',
                        '울산':'울산광역시',
                        '세종':'세종특별자치시',
                        '경기':'경기도',
                        '강원':'강원도',
                        '충북':'충청북도',
                        '충남':'충청남도',
                        '전북':'전라북도',
                        '전남':'전라남도',
                        '경북':'경상북도',
                        '경남':'경상남도',
                        '제주':'제주특별자치도'})

pop_df
# pop_df.info()

,행정구역,인구,인구밀도
1,서울특별시,"9,766","16,136"
2,부산광역시,"3,424","4,447"
3,대구광역시,"2,458","2,782"
4,인천광역시,"2,924","2,750"
5,광주광역시,"1,495","2,984"
6,대전광역시,"1,528","2,832"
7,울산광역시,"1,159","1,092"
8,세종특별자치시,266,571
9,경기도,"12,786","1,255"
10,강원도,"1,521",90


In [43]:
# 행정구역 이름 동일한지 확인
set(pop_df["행정구역"]) == set(df_2["시도별"])

True

In [44]:
# 두 데이터셋 합치기
nstore_pop_df = pd.merge(df_2, pop_df, left_on="시도별", right_on='행정구역', how='left') 
nstore_pop_df

,버거킹,KFC,맥도날드,롯데리아,BKM,버거지수,시도별,1인당 지역내총생산,1인당 지역총소득,1인당 개인소득,1인당 민간소비,행정구역,인구,인구밀도
0,9,3,7,54,19,0.351852,강원도,32061,29392,18997,16811,강원도,"1,521",90
1,100,50,96,317,246,0.776025,경기도,36133,38466,20482,17399,경기도,"12,786","1,255"
2,24,4,29,87,57,0.655172,경상남도,33690,32140,18939,16426,경상남도,"3,339",317
3,25,4,15,68,44,0.647059,경상북도,40272,34873,18611,16217,경상북도,"2,675",141
4,15,3,10,42,28,0.666667,광주광역시,27548,30964,20532,18231,광주광역시,"1,495","2,984"
5,20,10,25,66,55,0.833333,대구광역시,23744,27798,19210,17850,대구광역시,"2,458","2,782"
6,14,5,13,51,32,0.627451,대전광역시,28364,31548,20498,18025,대전광역시,"1,528","2,832"
7,20,9,41,82,70,0.853659,부산광역시,27409,29388,19680,18030,부산광역시,"3,424","4,447"
8,111,73,96,181,280,1.546961,서울특별시,44865,49121,23440,21891,서울특별시,"9,766","16,136"
9,0,1,0,10,1,0.100000,세종특별자치시,35826,36983,19789,16762,세종특별자치시,266,571


In [45]:
# 인구, 인구밀도 type : object → int
nstore_pop_df["인구"] = nstore_pop_df["인구"].str.replace(",","",regex=False)
nstore_pop_df["인구"] = nstore_pop_df["인구"].astype(int)

nstore_pop_df["인구밀도"] = nstore_pop_df["인구밀도"].str.replace(",","",regex=False)
nstore_pop_df["인구밀도"] = nstore_pop_df["인구밀도"].astype(int)

nstore_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   버거킹         17 non-null     int32  
 1   KFC         17 non-null     int32  
 2   맥도날드        17 non-null     int32  
 3   롯데리아        17 non-null     int32  
 4   BKM         17 non-null     int32  
 5   버거지수        17 non-null     float64
 6   시도별         17 non-null     object 
 7   1인당 지역내총생산  17 non-null     int64  
 8   1인당 지역총소득   17 non-null     int64  
 9   1인당 개인소득    17 non-null     int64  
 10  1인당 민간소비    17 non-null     int64  
 11  행정구역        17 non-null     object 
 12  인구          17 non-null     int32  
 13  인구밀도        17 non-null     int32  
dtypes: float64(1), int32(7), int64(4), object(2)
memory usage: 1.5+ KB


In [46]:
# 행정구역 컬럼 지우기
nstore_pop_df.drop("행정구역", axis=1, inplace=True)
nstore_pop_df

,버거킹,KFC,맥도날드,롯데리아,BKM,버거지수,시도별,1인당 지역내총생산,1인당 지역총소득,1인당 개인소득,1인당 민간소비,인구,인구밀도
0,9,3,7,54,19,0.351852,강원도,32061,29392,18997,16811,1521,90
1,100,50,96,317,246,0.776025,경기도,36133,38466,20482,17399,12786,1255
2,24,4,29,87,57,0.655172,경상남도,33690,32140,18939,16426,3339,317
3,25,4,15,68,44,0.647059,경상북도,40272,34873,18611,16217,2675,141
4,15,3,10,42,28,0.666667,광주광역시,27548,30964,20532,18231,1495,2984
5,20,10,25,66,55,0.833333,대구광역시,23744,27798,19210,17850,2458,2782
6,14,5,13,51,32,0.627451,대전광역시,28364,31548,20498,18025,1528,2832
7,20,9,41,82,70,0.853659,부산광역시,27409,29388,19680,18030,3424,4447
8,111,73,96,181,280,1.546961,서울특별시,44865,49121,23440,21891,9766,16136
9,0,1,0,10,1,0.100000,세종특별자치시,35826,36983,19789,16762,266,571


In [47]:
# csv, xlsx 파일로 내보내기

nstore_pop_df.to_csv(output_path+"nstore_stat_df.csv", index=False)
nstore_pop_df.to_excel(output_path+"nstore_stat_df.xlsx", index=False)

os.listdir(output_path)

['.DS_Store',
 'burger_df.csv',
 'burger_df.xlsx',
 'nstore_df.csv',
 'nstore_df.xlsx',
 'nstore_stat_df.csv',
 'nstore_stat_df.xlsx']